In [ ]:
# Plot optics, rays, celestial coords, and various official Rubin coordinate systems in 3D.
# Some notes: 
#   - "telescope parked position" is
#     • altitude = 90 degrees
#     • azimuth = 90 degrees  (i.e., telescope would point East if tipped off zenith)
#     • rotation = 0 degrees  
#   - batoid.globalCoordSys = ipvolume coordinate system = Observatory Mount Coordinate System (MCS)

In [ ]:
import batoid
import numpy as np
import plotly.graph_objects as go

In [ ]:
fiducial_telescope = batoid.Optic.fromYaml("LSST_r.yaml")
dist = 3000  # "distance" in m to Celestial sphere for plotting purposes

In [ ]:
def xfan(telescope, theta_x, theta_y, nx, **kwargs):
    return batoid.RayVector.asGrid(telescope, wavelength=625e-9, theta_x=theta_x, theta_y=theta_y, nx=nx, ny=1, **kwargs)
def yfan(telescope, theta_x, theta_y, ny, **kwargs):
    return batoid.RayVector.asGrid(telescope, wavelength=625e-9, theta_x=theta_x, theta_y=theta_y, nx=1, ny=ny, **kwargs)
def xyfan(telescope, theta_x, theta_y, n, **kwargs):
    rays1 = xfan(telescope, theta_x, theta_y, n, **kwargs)
    rays2 = yfan(telescope, theta_x, theta_y, n, **kwargs)
    return batoid.concatenateRayVectors([rays1, rays2])

In [ ]:
def plotline(fig, ctf, x, y, z):
    x, y, z = np.broadcast_arrays(x, y, z)
    x, y, z = ctf.applyForwardArray(x, y, z)
    ax.add_scatter3d(
        x=x, y=y, z=z,
        mode='lines',
        line=dict(color='red', width=1)
    )

In [ ]:
def draw3dFP(fig, coordSys):
    # Just has to be approximately right...
    raftsize = 0.31527*2/5
    ctf = batoid.CoordTransform(coordSys, batoid.globalCoordSys)
    
    # top hline
    plotlineipv(fig, ctf, np.array([-1.5, 1.5])*raftsize, 2.5*raftsize, 0)
    # middle hlines
    plotline(fig, ctf, np.array([-2.5, 2.5])*raftsize, 1.5*raftsize, 0)
    plotline(fig, ctf, np.array([-2.5, 2.5])*raftsize, 0.5*raftsize, 0)
    plotline(fig, ctf, np.array([-2.5, 2.5])*raftsize, -0.5*raftsize, 0)
    plotline(fig, ctf, np.array([-2.5, 2.5])*raftsize, -1.5*raftsize, 0)
    # bottom hline
    plotline(fig, ctf, np.array([-1.5, 1.5])*raftsize, -2.5*raftsize, 0)
    # left vline
    plotline(fig, ctf, -2.5*raftsize, np.array([-1.5, 1.5])*raftsize, 0)
    # middle vlines
    plotline(fig, ctf, -1.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    plotline(fig, ctf, -0.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    plotline(fig, ctf, 0.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    plotline(fig, ctf, 1.5*raftsize, np.array([-2.5, 2.5])*raftsize, 0)
    # right vline
    plotline(fig, ctf, 2.5*raftsize, np.array([-1.5, 1.5])*raftsize, 0)

In [ ]:
def drawSkyFP(fig, alt, az, rot):
    ctf = batoid.CoordTransform(
        batoid.globalCoordSys,
        batoid.CoordSys(
            (0, 0, 0),
            batoid.RotX(-rot)@batoid.RotY(alt)@batoid.RotZ(az)
        )
    )
    f = dist*np.deg2rad(0.7)

    # top hline
    plotline(fig, ctf, dist, np.array([-1.5, 1.5])*f, 2.5*f*np.ones(2))
    # middle hlines
    plotline(fig, ctf, dist, np.array([-2.5, 2.5])*f, 1.5*f*np.ones(2))
    plotline(fig, ctf, dist, np.array([-2.5, 2.5])*f, 0.5*f*np.ones(2))
    plotline(fig, ctf, dist, np.array([-2.5, 2.5])*f, -0.5*f*np.ones(2))
    plotline(fig, ctf, dist, np.array([-2.5, 2.5])*f, -1.5*f*np.ones(2))    
    # bottom hline
    plotline(fig, ctf, dist, np.array([-1.5, 1.5])*f, -2.5*f*np.ones(2))
    # left vline
    plotline(fig, ctf, dist, -2.5*f*np.ones(2), np.array([-1.5, 1.5])*f)
    # middle vlines
    plotline(fig, ctf, dist, -1.5*f*np.ones(2), np.array([-2.5, 2.5])*f)
    plotline(fig, ctf, dist, -0.5*f*np.ones(2), np.array([-2.5, 2.5])*f)
    plotline(fig, ctf, dist, 0.5*f*np.ones(2), np.array([-2.5, 2.5])*f)
    plotline(fig, ctf, dist, 1.5*f*np.ones(2), np.array([-2.5, 2.5])*f)
    # right vline
    plotline(fig, ctf, dist, 2.5*f*np.ones(2), np.array([-1.5, 1.5])*f)

In [ ]:
def drawAzimuthRing(fig):
    th = np.linspace(0, 2*np.pi, 100)
    x_, y_ = np.cos(th), np.sin(th)
    for d in [-0.1, 0.1]:
        for r in [4.5, 5.0]:
            x = x_*r
            y = y_*r
            z = d
            fig.scatter_3d(
                x=x, y=y, z=z, 
                mode='lines',
                line=dict(color='red', width=1)
            )

In [ ]:
def drawElevationBearings(ipv, az):
    th = np.linspace(0, 2*np.pi, 100)
    x_, y_ = np.cos(th), np.sin(th)
    for d in [4.4, 4.5]:
        for r in [1.5, 1.75]:
            x = -d
            y = x_*r
            z = y_*r            
            z += 5.425  # height of elevation axis above azimuth ring
            x, y = np.cos(np.pi/2-az)*x-np.sin(np.pi/2-az)*y, np.sin(np.pi/2-az)*x+np.cos(np.pi/2-az)*y
            
            ipv.plot(x, y, z, color='black')
            ipv.plot(-x, -y, z, color='black')

In [ ]:
def drawAltAzGrid(ipv):
    th = np.linspace(0, 2*np.pi, 100)
    for alt in np.arange(0, 81, 10):
        x = np.cos(th)*np.cos(np.deg2rad(alt))
        y = np.sin(th)*np.cos(np.deg2rad(alt))
        z = np.full_like(th, np.sin(np.deg2rad(alt)))
        ipv.plot(x*dist, y*dist, z*dist, color='green')
    th = np.linspace(0, np.pi/2, 50)
    for az in np.linspace(0, 2*np.pi, 12, endpoint=False):
        x = np.cos(az)*np.cos(th)
        y = np.sin(az)*np.cos(th)
        z = np.sin(th)
        if az == 0:
            ipv.plot(x*dist, y*dist, z*dist, color='red')
        else:
            ipv.plot(x*dist, y*dist, z*dist, color='green')

In [ ]:
def filter_xyz_horizon(x, y, z):
    w = z>0
    if np.any(w) and not np.all(w):
        indices = np.where(np.diff(w.astype(int))==1)[0]
        if len(indices) != 1:
            idx = 0
        else:
            idx = indices[0]+1
        w = np.roll(w, -idx)
        x = np.roll(x, -idx)
        y = np.roll(y, -idx)
        z = np.roll(z, -idx)
    return w, x, y, z

In [ ]:
def drawEqGrid(ipv, lat):
    # Need to rotate around +y axis by 90-lat
    ctf = batoid.CoordTransform(
        batoid.globalCoordSys,
        batoid.CoordSys(
            (0,0,0), 
            batoid.RotY(-(np.pi/2-lat))
        )
    )
    th = np.linspace(0, 2*np.pi, 100)
    for alt in np.arange(-90, 91, 10):
        x = np.cos(th)*np.cos(np.deg2rad(alt))
        y = np.sin(th)*np.cos(np.deg2rad(alt))
        z = np.full_like(th, np.sin(np.deg2rad(alt)))
        x, y, z = ctf.applyForwardArray(x, y, z)
        w, x, y, z = filter_xyz_horizon(x, y, z)
        if np.any(w):
            ipv.plot(x[w]*dist, y[w]*dist, z[w]*dist, color='blue')
    for az in np.linspace(0, 2*np.pi, 12, endpoint=False):
        x = np.cos(az)*np.cos(th)
        y = np.sin(az)*np.cos(th)
        z = np.sin(th)
        x, y, z = ctf.applyForwardArray(x, y, z)
        w, x, y, z = filter_xyz_horizon(x, y, z)
        if np.any(w):
            ipv.plot(x[w]*dist, y[w]*dist, z[w]*dist, color='blue')

In [ ]:
def getAzCS(az):
    """Return Azimuth Coordinate System (AzCS).  (LTS-136)

    AzCS is defined by:
        • +z towards zenith
        • Telescope at horizon points towards +y
        • +x completes RHR
        • Origin at center of azimuth ring.
        
    Parameters
    ----------
    az : float
        Azimuth in radians
    
    Returns
    -------
    batoid.CoordSys
    """
    return batoid.CoordSys(
        origin = [0, 0, 0],
        rot = batoid.RotZ(np.pi/2-az)
    )

In [ ]:
def getOpCS(alt, az):
    """Return Optical Coordinate System (OpCS).  (LTS-136)

    OpCS is defined by:
        • +x same as +x in AzCS
        • +z along boresight, towards sky
        • +y completes RHR        
        • Origin at M1 vertex
        
    Parameters
    ----------
    alt : float
        Altitude in radians
    az : float
        Azimuth in radians
    
    Returns
    -------
    batoid.CoordSys
    """
    # Must be a better way to do this than using the complete telescope,
    # but this works for now.
    telescope = fiducial_telescope
    telescope = telescope.withGlobalShift([0, 0, 3.53])  # Height of M1 vertex above azimuth ring
    telescope = telescope.withLocalRotation(batoid.RotZ(np.pi/2-az))
    telescope = telescope.withLocalRotation(batoid.RotX(np.pi/2-alt), rotOrigin=[0, 0, 5.425], coordSys=batoid.globalCoordSys)
    return telescope['M1'].coordSys

In [ ]:
def getMCS():
    """Draw axes for Observatory Mount Coordinate System (OCS).  (LCA-280)
    
    MCS is the same as ipv axes, or batoid.globalCoordSys
    
    Defined by:
        • +Z towards zenith
        • +X towards North
        • +Y from RHR (towards West)
        • Origin at center of azimuth ring.
        
    Returns
    -------
    batoid.CoordSys
    """
    return batoid.globalCoordSys

In [ ]:
def getTCS(alt, az):
    """Draw axes for Telescope Coordinate System (TCS).  (LCA-280)
        
    Defined by:
        • +Z towards sky along optic axis.  Follows telescope.
        • +X towards North when telescope parked.  Follows telescope.
        • +Y from RHR (towards West when parked).  Follows telescope.
        • Origin at M1 vertex

    Parameters
    ----------
    alt : float
        Altitude in radians
    az : float
        Azimuth in radians

    Returns
    -------
    batoid.CoordSys
    """
    return getOpCS(alt, az)

In [ ]:
def drawCoordSys(ipv, coordSys, length=2):
    p0 = coordSys.origin
    px = p0 + coordSys.rot @ (np.array([1, 0, 0]) * length)
    py = p0 + coordSys.rot @ (np.array([0, 1, 0]) * length)
    pz = p0 + coordSys.rot @ (np.array([0, 0, 1]) * length)
    ipv.plot(*np.vstack([p0, px]).T, color='blue')
    ipv.plot(*np.vstack([p0, py]).T, color='red')
    ipv.plot(*np.vstack([p0, pz]).T, color='black')

In [ ]:
def show_coord_sys(rot, alt, az, thx, thy, doAzCS=False, doOpCS=False, doMCS=False, doTCS=False, doCCS=False):
    lat = np.deg2rad(-30.2446)
    telescope = fiducial_telescope
    telescope = telescope.withGlobalShift([0, 0, 3.53])  # Height of M1 vertex above azimuth ring
    telescope = telescope.withLocallyRotatedOptic("LSSTCamera", batoid.RotZ(rot))
    telescope = telescope.withLocalRotation(batoid.RotZ(np.pi/2-az))
    telescope = telescope.withLocalRotation(batoid.RotX(np.pi/2-alt), rotOrigin=[0, 0, 5.425], coordSys=batoid.globalCoordSys)

    # rays = xyfan(telescope, thx, thy, 30, backDist=dist/10)
    rays = xyfan(telescope, thx, thy, 30)
    traceFull = telescope.traceFull(rays)

    fig = go.Figure(
        layout=dict(
            showlegend=False,
            width=800,
            height=800
        ),
    )
    
    # drawAzimuthRing(ipv)
    # drawElevationBearings(ipv, az)    
    telescope.draw3d(fig, plotly=True, mode='lines', line=dict(color='black'))

    batoid.drawTrace3d(fig, traceFull, plotly=True, mode='lines', line=dict(color='blue'))

    # extent rays all the way back to the Celestial sphere
    w = ~traceFull['M1']['out'].vignetted
    rays = rays.toCoordSys(batoid.globalCoordSys)
    p0 = rays[w].r
    p1 = rays[w].positionAtTime(-dist)
    for _p0, _p1 in zip(p0, p1):
        fig.add_scatter3d(
            x=[_p0[0], _p1[0]],
            y=[_p0[1], _p1[1]],
            z=[_p0[2], _p1[2]],
            mode='lines',
            line=dict(color='blue')
        )
    camera = dict(
        up=dict(x=0, y=0, z=1),
        center=dict(x=0, y=0, z=0),
        eye=dict(x=1, y=1, z=1)
    )
    fig.update_layout(
        scene_camera=camera,
        scene = dict(
            xaxis = dict(visible=False, range=[-300, 300]),
            yaxis = dict(visible=False, range=[-300, 300]),
            zaxis = dict(visible=False, range=[-300, 300]),
            xaxis_showspikes=False,
            yaxis_showspikes=False,
            zaxis_showspikes=False,
            aspectmode='cube',
            aspectratio=dict(x=1, y=1, z=1)
        ),
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0,
            pad=4
        ),
    )
#     draw3dFP(ipv, telescope['Detector'].coordSys)
#     drawSkyFP(ipv, alt, az, rot)
    
#     drawAltAzGrid(ipv)
#     drawEqGrid(ipv, lat)

#     if doAzCS:
#         AzCS = getAzCS(az)
#         AzCS = batoid.CoordSys(
#             origin = [0, 0, 0],
#             rot = batoid.RotZ(np.pi/2-az)
#         )
#         drawCoordSys(ipv, AzCS)
#     if doOpCS:
#         OpCS = getOpCS(alt, az)
#         drawCoordSys(ipv, OpCS)                         
#     if doMCS:
#         MCS = getMCS()
#         drawCoordSys(ipv, MCS)
#     if doTCS:
#         TCS = getTCS(alt, az)
#         drawCoordSys(ipv, TCS)
#     if doCCS:
#         drawCoordSys(ipv, telescope['LSSTCamera'].coordSys)            
            
#     ipv.show()
#     ipv.xlim(-8, 8)
#     ipv.ylim(-8, 8)
#     ipv.zlim(0, 16)
#     ipvfig.camera.position = (-1.5254697593501128, -1.2928322819658806, 0.03907305996705254)
#     ipvfig.camera.rotation = (-1.2237988550857888, -1.3392070664612021, -2.763432436269493, 'XYZ')
    fig.show()

In [ ]:
# rot = 0
rot = 0.7485759992183523 
alt = 0.8010022880203016 
az = 1.8888812762063498 
# az = 0

thx = np.deg2rad(0.0)
thy = np.deg2rad(0.0)
show_coord_sys(rot, alt, az, thx, thy, doCCS=True)